Yahoo finance (n)

Proszę wstawić print screen z np. jupyter notebooka, gdzie:

- będą widoczne 2 wykresy ze średnią ruchomą (pandas, altair)

- będzie widoczny kod, który wygenerował wykresy.


Nazwa pliku (bez polskich znaków):

Nazwisko_Imie_yahoo_niestacj.jpg

Dane:
    
https://finance.yahoo.com/quote/CSV/history/?guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAAJjgOLcqSa79X2NVcDfaAsiL9BAHg5RvrFg6uir9MoKK8kw1oCE7Io1rINBuyWaJzgkx4TiRiOL_UMQKHk9Lp2Ne9IZ7hwh-3ettI5wsQEvlI2guv04Y0DrmkdAYNKm_baNDrq-DX0kN7r07wdmeKavlvqjoWZWgGPyncARRyqtG&guccounter=2

In [1]:
import re

import pandas as pd
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def get_local_csv(path="houses_data.csv"):
    """
    Loads csv file into my_data variable and displays it's head.
        Parameters:
            path (str): Path to local file with extension. Must be '.csv'. Default = 'houses_data.csv'
        Returns:
            my_data (DataFrame): Global variable that holds loaded DataFrame.
    """
    
    global my_data
    
    if not re.search(r"(.csv)", path):
        raise SyntaxError("Only .csv are files allowed")
    else:
        my_data = pd.read_csv(path)
        
    return display(my_data.head())

In [5]:
def smaller_df(source=my_data, x="Date", y="Price", y_rolling=True, x_as_index=True) -> pd.DataFrame:
    """
    Reduces taken DataFrame to two columns (x, y) and calculates rolling mean for y values. 
    Could set x as index.
        Parameters:
            source (DataFrame): DataFrame with 2 and more columns. Default = my_data.
            x (array-like): Column name from source paremeter. Default = 'Date'.
            y (array-like): Column name from source paremeter. Default = 'Price'.
            y_rolling (bool): Calculates rolling mean for y values. Default = True.
            x_as_index (bool): If True, x must be datetime type. Default = True.  
        Returns:
            small_df (DataFrame): Global variable that holds calculated DataFrame.
    """
    
    global small_df
    
    small_df = pd.DataFrame()
    small_df[x] = source[x]
    small_df[y] = source[y]
    
    if y_rolling == True:
        small_df["Rolling"] = source[y].rolling(window=10).mean()
    
    if x_as_index == True:
        small_df.set_index(x, inplace=True)
    
    return small_df

In [7]:
def plot_pandas_rolling_mean(source=small_df, title="title") -> plt:
    """ 
    Plots columns of taken DataFrame with scatter (first column) and line plot (second column). 
    Could set title.
        Parameters:
            source (DataFrame): DataFrame with max 2 columns. Index must be date. Default = small_df.
            title (str): Title of plot. Default = 'title'.
        Returns: Plot.
    """
    
    plt.figure(figsize=((13,8)))
    
    sns.scatterplot(x=source.index, y=source.iloc[:,0], color="red", label=small_df.columns[0])
    sns.lineplot(x=source.index, y=source.iloc[:,1], color="blue", label=small_df.columns[1])
    
    plt.title(title)
    plt.xticks(rotation=90)
    plt.grid();
    
    return plt.show()

In [15]:
def plot_alt_rolling_mean(source=small_df):
    """
    Plots columns of taken DataFrame with scatter (first column) and line plot (second column). 
    Could set title.
        Parameters:
            source (DataFrame): DataFrame with max 2 columns. One of them must be date. Default = small_df.
        Returns: Plot.
    """
    alt.data_transformers.disable_max_rows()
    
    line = alt.Chart(source).mark_line(
        color='red',
        size=3
    ).transform_window(
        rolling_mean='mean(Price)',
        frame=[-10]
    ).encode(
        x='Date:T',
        y='rolling_mean:Q'
    )

    points = alt.Chart(source).mark_point().encode(
        x='Date:T',
        y=alt.Y('Price:Q',
                axis=alt.Axis(title='Prices'))
    ).properties(
    width=640,
    height=480)

    return points + line

In [17]:
def main():
    get_local_csv()
    
    smaller_df(y="Price", x_as_index=True)
    
    plot_pandas_rolling_mean(title="Prices of yahoo houses and rolling mean")
    
    smaller_df(y="Price", x_as_index=False, y_rolling=False)
    
    plot_alt_rolling_mean()